In [3]:
# %load ~/ipyhead
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline
import seaborn as sns

In [4]:
!pip install pyensembl

In [5]:
!pyensembl install --release 79 --species human

-- Running 'install' for EnsemblRelease(release=79, species='homo_sapiens')
INFO:root:Loaded sequence dictionary from /home/jacquelineburos/.cache/pyensembl/GRCh38/ensembl79/Homo_sapiens.GRCh38.cdna.all.fa.gz.pickle
INFO:root:Loaded sequence dictionary from /home/jacquelineburos/.cache/pyensembl/GRCh38/ensembl79/Homo_sapiens.GRCh38.pep.all.fa.gz.pickle


In [6]:
from pyensembl import cached_release

download datasets and process:

In [7]:
!gsutil ls gs://mz-hammerlab/output/

gs://mz-hammerlab/output/
gs://mz-hammerlab/output/ERR431566.logs.tar.gz
gs://mz-hammerlab/output/ERR431566.tar.gz
gs://mz-hammerlab/output/ERR431567.logs.tar.gz
gs://mz-hammerlab/output/ERR431567.tar.gz
gs://mz-hammerlab/output/ERR431568.logs.tar.gz
gs://mz-hammerlab/output/ERR431568.tar.gz
gs://mz-hammerlab/output/ERR431569.logs.tar.gz
gs://mz-hammerlab/output/ERR431569.tar.gz
gs://mz-hammerlab/output/ERR431570.logs.tar.gz
gs://mz-hammerlab/output/ERR431570.tar.gz
gs://mz-hammerlab/output/ERR431571.logs.tar.gz
gs://mz-hammerlab/output/ERR431571.tar.gz
gs://mz-hammerlab/output/ERR431572.logs.tar.gz
gs://mz-hammerlab/output/ERR431572.tar.gz
gs://mz-hammerlab/output/ERR431573.logs.tar.gz
gs://mz-hammerlab/output/ERR431573.tar.gz
gs://mz-hammerlab/output/ERR431574.logs.tar.gz
gs://mz-hammerlab/output/ERR431574.tar.gz
gs://mz-hammerlab/output/ERR431575.logs.tar.gz
gs://mz-hammerlab/output/ERR431575.tar.gz
gs://mz-hammerlab/output/ERR431576.logs.tar.gz
gs://mz-hammerlab/output/ERR431576.ta

In [8]:
!gsutil cp gs://mz-hammerlab/output/ERR431624.tar.gz .;
!gsutil cp gs://mz-hammerlab/output/ERR431624.logs.tar.gz .;
!tar -zxvf ERR431624.tar.gz && rm ERR431624.tar.gz;
!tar -zxvf ERR431624.logs.tar.gz && rm ERR431624.logs.tar.gz;

Copying gs://mz-hammerlab/output/ERR431624.tar.gz...
Copying gs://mz-hammerlab/output/ERR431624.logs.tar.gz...
output/ERR431624/
output/ERR431624/abundance.tsv
output/ERR431624/abundance.h5
output/ERR431624/run_info.json
logs/ERR431624/out.txt
logs/ERR431624/sar.txt
logs/ERR431624/time.txt


In [9]:
def download(s):
    !gsutil cp gs://mz-hammerlab/output/"$s".tar.gz .;
    !gsutil cp gs://mz-hammerlab/output/"$s".logs.tar.gz .;
    !tar -zxvf "$s".tar.gz && rm "$s".tar.gz;
    !tar -zxvf "$s".logs.tar.gz && rm "$s".logs.tar.gz;
    print 'downloaded', s
download('ERR431583')

Copying gs://mz-hammerlab/output/ERR431583.tar.gz...
Copying gs://mz-hammerlab/output/ERR431583.logs.tar.gz...
output/ERR431583/
output/ERR431583/abundance.tsv
output/ERR431583/abundance.h5
output/ERR431583/run_info.json
logs/ERR431583/out.txt
logs/ERR431583/sar.txt
logs/ERR431583/time.txt
downloaded ERR431583


In [10]:
!ls -l output

total 8
drwxr-xr-x 2 jacquelineburos jacquelineburos 4096 Jul 29 23:40 ERR431583
drwxr-xr-x 2 jacquelineburos jacquelineburos 4096 Jul 31 19:53 ERR431624


In [11]:
from pyensembl import cached_release
ensembl_release = cached_release(79)
def load_multiple(files):
    """
    files is ordered list of samples. they will be assigned sample ids 1-n.
    """
    dfs = []
    for ix, f in enumerate(files):
        df = pd.read_csv('data/output/%s/abundance.tsv' % f, sep='\t')
        df['sample_id'] = ix+1
        df['filename'] = f
        df['gene_name'] = df['target_id'].map(lambda t: ensembl_release.gene_name_of_transcript_id(t))
        dfs.append(df)
    return pd.concat(dfs)

def simple_summary(df):
    """
    sum counts, abundance (tpm) across genes
    """
    return df.groupby(['sample_id', 'filename', 'gene_name'])[['est_counts', 'tpm']].sum().reset_index()

In [12]:
# process both B cells
simple_summary = simple_summary(load_multiple(['ERR431572', 'ERR431624', 'ERR431583']))
simple_summary.head()

,sample_id,filename,gene_name,est_counts,tpm
0,1,ERR431572,A1BG,80.232280,11.155641
1,1,ERR431572,A1CF,18.042401,0.382263
2,1,ERR431572,A2M,3.000000,1.226161
3,1,ERR431572,A2ML1,21.912401,2.756862
4,1,ERR431572,A2MP1,2.000000,0.092441


In [13]:
simple_summary.tail()

,sample_id,filename,gene_name,est_counts,tpm
104491,3,ERR431583,bP-2189O9.1,0.000000,0.000000
104492,3,ERR431583,bP-2189O9.2,3.263100,0.347490
104493,3,ERR431583,bP-2189O9.4,0.000000,0.000000
104494,3,ERR431583,pk,63.556597,2.080726
104495,3,ERR431583,yR211F11.2,2.000000,0.715197


In [14]:
simple_summary.to_csv('data/summary.simple.tsv', sep='\t', index=None)        

## A quick look at all the datafiles we have

In [15]:
datafiles = pd.read_csv('data_filenames.tsv', sep='\t')
datafiles[['filename', 'SubSet', 'Antibody']]

,filename,SubSet,Antibody
0,ERR431599,B_CD5,"CD19+,CD5+"
1,ERR431596,B_CD5,"CD19+,CD5+"
2,ERR431591,B_CD5,"CD19+,CD5+"
3,ERR431568,B_CD5,"CD19+,CD5+"
4,ERR431617,B_Memory,"CD19+,CD5-,CD27+"
5,ERR431569,B_Memory,"CD19+,CD5-,CD27+"
6,ERR431594,B_Memory,"CD19+,CD5-,CD27+"
7,ERR431613,B_Memory,"CD19+,CD5-,CD27+"
8,ERR431608,B_Memory,"CD19+,CD5-,CD27+"
9,ERR431572,B_Naive,"CD19+,CD5-,CD27-"


In [16]:
datafiles[datafiles.SubSet.str.lower().str.contains('treg')]

,Source,Name,SubSet,Antibody,InnerSize,R1 URI,R1 MD5SUM,R2 URI,R2 MD5SUM,filename
44,SQ_0021,ERS403436,CD4_Treg,"CD4+,CD127-,CD25+",450,ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR431/ERR4...,1f30880aa773dcc912b2e7c6db2ae6fe,ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR431/ERR4...,c5d9d48830eb5d183b6ce080b6a329b2,ERR431583
45,SQ_0022,ERS403386,CD4_Treg,"CD4+,CD127-,CD25+",450,ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR431/ERR4...,b9fd31742d7a4eff1790982a64795146,ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR431/ERR4...,ebc1489184defb91e5b8814123413152,ERR431606
46,SQ_0023,ERS403435,CD4_Treg,"CD4+,CD127-,CD25+",450,ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR431/ERR4...,c3435337bcee5dad1524d6af609f47f3,ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR431/ERR4...,66bf895e7b84440297bea97b2aaa07e1,ERR431609
47,SQ_0065,ERS403410,CD4_Treg,"CD4+,CD127-,CD25+",449,ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR431/ERR4...,8aae8956187b8fdfd0ba12701feb173b,ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR431/ERR4...,6b42df8aa6924553d2f40de84eead826,ERR431601
48,SQ_0067,ERS403428,CD4_Treg,"CD4+,CD127-,CD25+",444,ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR431/ERR4...,d36596f9f35d041225666659e6494cf7,ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR431/ERR4...,8fa326a4a158bcec4c5e9adc2f12316c,ERR431589


In [18]:
metadata = datafiles[['filename', 'SubSet', 'Antibody']]
metadata.head()

,filename,SubSet,Antibody
0,ERR431599,B_CD5,"CD19+,CD5+"
1,ERR431596,B_CD5,"CD19+,CD5+"
2,ERR431591,B_CD5,"CD19+,CD5+"
3,ERR431568,B_CD5,"CD19+,CD5+"
4,ERR431617,B_Memory,"CD19+,CD5-,CD27+"


In [19]:
## inspired by 
## http://stackoverflow.com/questions/17116814/pandas-how-do-i-split-text-in-a-column-into-multiple-rows
def split_rows_by(df, field, suffix='', by=','):
    s = df[field].str.split(by).apply(pd.Series, 1).stack()
    s.index = s.index.droplevel(-1)
    s.name = '{}{}'.format(field, suffix)
    if suffix == '':
        del df[field]
    return df.join(s)


In [144]:
## reshape metadata to list antibody classes as features
## version 1
df = metadata.copy()
subsets = split_rows_by(df, 'Antibody')
del subsets['SubSet']
subsets['value'] = 1
subsets.drop_duplicates(['filename','Antibody', 'value'], inplace=True)
subsets = subsets.pivot('filename', 'Antibody', 'value')

#subsets.reset_index(inplace=True)
#subsets.index = subsets.index.droplevel(-1)
subsets.head()


Antibody,CCR6+,CCR7+,CCR7-,CD127-,CD161+,CD19+,CD25+,CD27+,CD27-,CD4+,CD45RA+,CD45RA-,CD45RO+,CD45RO-,CD5+,CD5-,CD8+,CRTH2+,CXCR3+,CXCR3-
filename,,,,,,,,,,,,,,,,,,,,
ERR431566,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0
ERR431567,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN
ERR431568,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
ERR431569,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN
ERR431570,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN


In [165]:
## reshape metadata to list antibody classes as features
df = metadata.copy()
subsets = split_rows_by(df, 'Antibody')
subsets['antibody'] = subsets.Antibody.replace(to_replace=ur'.$', value='', inplace=False, regex=True)
subsets['value'] = subsets.Antibody.replace(to_replace=ur'^.+(.)$', value=ur'\1', inplace=False, regex=True)
del subsets['Antibody']
del subsets['SubSet']
subsets.drop_duplicates(['filename','antibody', 'value'], inplace=True)
subsets = subsets.pivot('filename', 'antibody', 'value')
subsets.reset_index(inplace=True)
subsets = pd.merge(subsets, df.loc[:,['filename','SubSet']], on='filename', how='outer')
subsets.head()

antibody,filename,CCR6,CCR7,CD127,CD161,CD19,CD25,CD27,CD4,CD45RA,CD45RO,CD5,CD8,CRTH2,CXCR3,SubSet
0,ERR431566,None,None,None,None,None,None,None,+,None,None,None,None,+,-,CD4_Th2
1,ERR431567,None,-,None,None,None,None,None,None,-,+,None,+,None,None,CD8_Effector
2,ERR431568,None,None,None,None,+,None,None,None,None,None,+,None,None,None,B_CD5
3,ERR431569,None,None,None,None,+,None,+,None,None,None,-,None,None,None,B_Memory
4,ERR431570,None,None,None,None,None,None,None,+,None,None,None,None,None,+,CD4_Th1


In [166]:
simple_summary.head()

,sample_id,filename,gene_name,est_counts,tpm
0,1,ERR431572,A1BG,80.232280,11.155641
1,1,ERR431572,A1CF,18.042401,0.382263
2,1,ERR431572,A2M,3.000000,1.226161
3,1,ERR431572,A2ML1,21.912401,2.756862
4,1,ERR431572,A2MP1,2.000000,0.092441


In [167]:

def annotated_summary(df, metadata, **kwargs):
    """
    sum counts, abundance (tpm) across genes
    """
    return pd.merge(df, metadata, on='filename', how='left', **kwargs)


In [168]:
annotated_summary = annotated_summary(simple_summary, metadata=subsets)

In [169]:
annotated_summary.head()

,sample_id,filename,gene_name,est_counts,tpm,CCR6,CCR7,CD127,CD161,CD19,CD25,CD27,CD4,CD45RA,CD45RO,CD5,CD8,CRTH2,CXCR3,SubSet
0,1,ERR431572,A1BG,80.232280,11.155641,None,None,None,None,+,None,-,None,None,None,-,None,None,None,B_Naive
1,1,ERR431572,A1CF,18.042401,0.382263,None,None,None,None,+,None,-,None,None,None,-,None,None,None,B_Naive
2,1,ERR431572,A2M,3.000000,1.226161,None,None,None,None,+,None,-,None,None,None,-,None,None,None,B_Naive
3,1,ERR431572,A2ML1,21.912401,2.756862,None,None,None,None,+,None,-,None,None,None,-,None,None,None,B_Naive
4,1,ERR431572,A2MP1,2.000000,0.092441,None,None,None,None,+,None,-,None,None,None,-,None,None,None,B_Naive


## look at CD19+ B cell group

In [176]:
annotated_summary.loc[annotated_summary['CD19']=='+',:].groupby('SubSet').count()

,sample_id,filename,gene_name,est_counts,tpm,CCR6,CCR7,CD127,CD161,CD19,CD25,CD27,CD4,CD45RA,CD45RO,CD5,CD8,CRTH2,CXCR3
SubSet,,,,,,,,,,,,,,,,,,,
B_Naive,69664,69664,69664,69664,69664,0,0,0,0,69664,0,69664,0,0,0,69664,0,0,0


In [ ]:
sns.boxplot(data=annotated_summary.loc[annotated_summary['CD19']=='+',:], x='gene_name', y='tpm')

In [ ]:
sns.boxplot(data=annotated_summary.loc[annotated_summary['CD19']=='+',:], x='gene_name', y='tpm', color='SubSet')

In [179]:
sns.jointplot(data=annotated_summary.loc[annotated_summary['CD19']=='+',:], x='tpm', color='SubSet', hist=False, kind='distplot')

TypeError: jointplot() takes at least 2 arguments (4 given)